In [ ]:
'''
def calcUtility(p, i):
    t = (p.sum() / np.sqrt((p*p).sum()) ) * np.sqrt((250/i))
    u = min(max(t, 0), 6) * p.sum()
    return u


date = dlabels.loc[y.index, 'date'].values
weight = dlabels.loc[y.index, 'weight'].values
resp = dlabels.loc[y.index, 'resp'].values
action = estimator.predict(X)

p_i = np.bincount(date, weight * resp * action)
t = p_i.sum() / np.sqrt((p_i ** 2).sum()) * np.sqrt(250 / p_i.size)
u = np.clip(t, 0, 6) * p_i.sum()

return u * scale
'''

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

def nulls(df):
    unitrainvals = df.isnull().sum().values.tolist()
    traincols = df.columns.values.tolist()
    nUni = list(zip(unitrainvals, traincols))
    print(*nUni, sep = "\n") 

In [ ]:
train_raw = reduce_mem_usage(pd.read_csv('../input/jane-street-market-prediction/train.csv'))

In [ ]:
train_raw.shape

In [ ]:
import gc
gc.collect()

In [ ]:
train = train_raw.fillna(0)

train['action'] = ((train['weight'].values * train['resp'].values) > 0).astype('int')

In [ ]:
nulls(train)

In [ ]:
drop_cols = ['date', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp', 'ts_id', 'weight']

In [ ]:
train.drop(drop_cols, inplace=True, axis=1)

In [ ]:
#val = train.iloc[2000000: , :]
#train = train.iloc[:2000000 , :]

#gc.collect()

In [ ]:
#val

In [ ]:
x = train.iloc[:, :130]
y = train.iloc[:, 130:]

In [ ]:
from sklearn import metrics

In [ ]:
from sklearn.model_selection  import StratifiedKFold
import time

In [ ]:
%%time
err = [] 
#y_pred_tot_lgm = np.zeros((len(test), 2))


fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=2020)
i = 1

for train_index, test_index in fold.split(x, y):
    x_train, x_val = x.iloc[train_index], x.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    m = CatBoostClassifier(n_estimators=10000,
                       #random_state=150303,
                       eval_metric='AUC',
                       learning_rate=0.08,
                       #depth=8,
                       #bagging_temperature=0.3,
                       task_type='GPU',
                           loss_function='Logloss',
                           use_best_model=True,
                           random_seed=6006,
                           metric_period=1
                       #num_leaves=64
                       
                       )
    m.fit(x_train, y_train,eval_set=[(x_val, y_val)], early_stopping_rounds=100,verbose=200)
    pred_y = m.predict(x_val)
    print(i, " err_lgm: ", metrics.accuracy_score(y_val,pred_y))
    err.append(metrics.roc_auc_score(y_val,pred_y))
    #y_pred_tot_lgm+= m.predict_proba(test)
    i = i + 1
#y_pred_tot_lgm=y_pred_tot_lgm/10
sum(err)/10

In [ ]:
#model = CatBoostClassifier(loss_function = 'Logloss', 
 #                          eval_metric='AUC', 
  #                       task_type="GPU",
   #                        use_best_model=True,
    #                       metric_period=1,
     #                      random_seed=6006)

In [ ]:
#model.fit(train.iloc[:, :130], train.iloc[:, 130:], eval_set=(val.iloc[:, :130], val.iloc[:, 130:]))

In [ ]:
drop_cols = ['date', 'weight']

In [ ]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set



In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    X_test = test_df.drop(drop_cols, axis=1)
    X_test = X_test.fillna(0)
    y_preds = m.predict(X_test)
    sample_prediction_df.action = y_preds
    env.predict(sample_prediction_df)